In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
tweets_file = open('all_on_30th_nov.txt', "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [12]:
len(tweets_data)

411703

In [13]:
from textblob import TextBlob

In [14]:
tweet_ds = []

In [15]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

In [16]:
from nltk.corpus import stopwords

In [17]:
stopWords = set(stopwords.words("english"))

In [18]:
def filter(tweet_text):
    if len(tweet_text) > 0:
        tokens = tokenizer.tokenize(tweet_text)
        tokens_filtered = []
        for i in tokens:
            if i.startswith('@') or i.startswith('#') or i.startswith('http'):
                if i in stopWords:
                    continue
            else:
                tokens_filtered.append(i.lower())
        return tokens_filtered

In [19]:
import re

In [20]:
for i in range(len(tweets_data)):
    try:
        if tweets_data[i]['lang'] == 'en':
            tweet_text = tweets_data[i]['text']
            tweet_edit = re.sub(r'[^\w\s]','',tweet_text)
            tokens = filter(tweet_edit)
            t = TextBlob(tweet_text)
            sentiment = t.sentiment.polarity
            tweet_ds.append([tweet_text, tokens, sentiment])
        else:
            continue
    except KeyError:
        continue

In [21]:
data = pd.DataFrame(tweet_ds, columns = {'tweet', 'tokens', 'sentiment'})

In [42]:
data.columns

Index(['tweet', 'sentiment', 'tokens'], dtype='object')

In [48]:
data = data.rename(columns = {'tweet':'tweet','sentiment':'tokens','tokens':'sentiment'})

In [22]:
dir()

['In',
 'Out',
 'TextBlob',
 'TweetTokenizer',
 '_',
 '_12',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i12',
 '_i13',
 '_i14',
 '_i15',
 '_i16',
 '_i17',
 '_i18',
 '_i19',
 '_i2',
 '_i20',
 '_i21',
 '_i22',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 '_sh',
 'data',
 'exit',
 'filter',
 'get_ipython',
 'i',
 'json',
 'line',
 'pd',
 'plt',
 'quit',
 're',
 'sentiment',
 'stopWords',
 'stopwords',
 't',
 'tokenizer',
 'tokens',
 'tweet',
 'tweet_ds',
 'tweet_edit',
 'tweet_text',
 'tweets_data',
 'tweets_file']

### Deleting variables to free up memory

In [26]:
del tweets_data

In [27]:
del tweet_ds

In [30]:
del tweet

In [32]:
del tokenizer

In [34]:
del sentiment

In [35]:
del stopwords

In [36]:
del stopWords

### Loading dependencies for our classification model

In [37]:
from __future__ import absolute_import, division, print_function

In [38]:
import codecs, multiprocessing, os, pprint, re

In [39]:
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

C:\Users\Gaurav Ahlawat\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [40]:
import gensim
LabeledSentence = gensim.models.doc2vec.LabeledSentence

In [ ]:
## !pip install keras

In [4]:
## data = pd.read_csv('dl_data_for_sentiment.csv', encoding = "ISO-8859-1")

In [49]:
data.tail()

,tweet,tokens,sentiment
320514,RT @NK_ESQ: So no FA interest in Wenger's comm...,"[rt, nk_esq, so, no, fa, interest, in, wengers...",0
320515,@PaulAhdal @GazetteBoro @PTallentireGaz ever E...,"[paulahdal, gazetteboro, ptallentiregaz, ever,...",0
320516,"""Guns from the Vanguard arsenal are great. The...","[guns, from, the, vanguard, arsenal, are, grea...",0.483333
320517,#YIAYwar Captan América will see the city and ...,"[yiaywar, captan, américa, will, see, the, cit...",0
320518,Quite an interesting thing about Ashley Young....,"[quite, an, interesting, thing, about, ashley,...",0.3


In [50]:
data.tokens[0]

['rt',
 'johnryan',
 '89',
 'oh',
 'mané',
 'mané',
 '3',
 'points',
 'important',
 'with',
 'some',
 'of',
 'the',
 'big',
 'boys',
 'taking',
 'points',
 'off',
 'each',
 'other',
 'tomorrow',
 'lfc',
 'whuliv',
 'btsport']

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [52]:
X_train, X_test, y_train, y_test = train_test_split(data.tokens, data.sentiment, test_size = 0.15)

In [53]:
def labelize_tweet(tweets, label_type):
    labelized = []
    for i, v in enumerate(tweets):
        label = '%s_%s' %(label_type, i)
        labelized.append(LabeledSentence(v,[label]))
    return labelized

X_train = labelize_tweet(X_train, 'train')
X_test = labelize_tweet(X_test, 'test')

In [54]:
X_train[0]

LabeledSentence(words=['what', 'a', 'save', 'from', 'the', 'spurs', 'keeper'], tags=['train_0'])

## Loading a pre-trained word2vec model

In [55]:
tweets2vec = w2v.Word2Vec.load(os.path.join("trained2", "tweets2vec.w2v"))

### Just testing whether it works or not

In [56]:
tweets2vec.similarity('united', 'chelsea')

0.93291073010698011

In [57]:
vectorizer = TfidfVectorizer(analyzer = lambda x:x, min_df = 10)

In [58]:
matrix = vectorizer.fit_transform([x.words for x in X_train])

In [62]:
tfidf = dict(zip(vectorizer.get_feature_names(),vectorizer.idf_))

In [63]:
print('vocab size :', len(tfidf))

vocab size : 13252


In [64]:
def build_word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweets2vec[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

### Creating weighted averages of word vectors using Tf-IDf scores on word embeddings

In [66]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([build_word_vector(z, 200) for z in (map(lambda x: x.words, X_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([build_word_vector(z, 200) for z in (map(lambda x: x.words, X_test))])
test_vecs_w2v = scale(test_vecs_w2v)

# Starting Deep Learning

In [69]:
!pip install tensorflow

  Running setup.py bdist_wheel for markdown: started
  Running setup.py bdist_wheel for markdown: finished with status 'done'
  Stored in directory: C:\Users\Gaurav Ahlawat\AppData\Local\pip\Cache\wheels\bf\46\10\c93e17ae86ae3b3a919c7b39dad3b5ccf09aeb066419e5c1e5
  Running setup.py bdist_wheel for html5lib: started
  Running setup.py bdist_wheel for html5lib: finished with status 'done'
  Stored in directory: C:\Users\Gaurav Ahlawat\AppData\Local\pip\Cache\wheels\6f\85\6c\56b8e1292c6214c4eb73b9dda50f53e8e977bf65989373c962
Successfully built markdown html5lib
  Found existing installation: html5lib 0.999
    Uninstalling html5lib-0.999:
      Successfully uninstalled html5lib-0.999


    DEPRECATION: Uninstalling a distutils installed project (html5lib) has been deprecated and will be removed in a future version. This is due to the fact that uninstalling a distutils project will only partially uninstall the project.


In [72]:
from keras.models import Sequential
model = Sequential()

In [74]:
from keras.layers import Dense

### This simple two layer model is showing some really bad accuracy!

## The reason might be the low number of tweets, which are around 300k, out of which I suspect some have sentiment equal to 0, let's check it out, but before that, lets try and see if increasing the complexity of the model by adding new layers has any substantial impact on the accuracy

In [75]:
model.add(Dense(32, activation='relu', input_dim=200))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_vecs_w2v, y_train, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
 - 12s - loss: 0.2342 - acc: 0.4449
Epoch 2/10
 - 11s - loss: 0.1755 - acc: 0.4453
Epoch 3/10
 - 10s - loss: 0.1450 - acc: 0.4456
Epoch 4/10
 - 11s - loss: 0.1245 - acc: 0.4462
Epoch 5/10
 - 11s - loss: 0.1104 - acc: 0.4461
Epoch 6/10
 - 10s - loss: 0.0994 - acc: 0.4464
Epoch 7/10
 - 11s - loss: 0.0903 - acc: 0.4463
Epoch 8/10
 - 11s - loss: 0.0825 - acc: 0.4463
Epoch 9/10
 - 11s - loss: 0.0760 - acc: 0.4467
Epoch 10/10
 - 12s - loss: 0.0702 - acc: 0.4470


## No effect at all!!!

In [76]:
model1 = Sequential()
model1.add(Dense(32, activation='relu', input_dim=200))
model1.add(Dense(32, activation='relu', input_dim=200))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_vecs_w2v, y_train, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
 - 11s - loss: 0.0663 - acc: 0.4471
Epoch 2/10
 - 11s - loss: 0.0607 - acc: 0.4470
Epoch 3/10
 - 10s - loss: 0.0575 - acc: 0.4474
Epoch 4/10
 - 11s - loss: 0.0532 - acc: 0.4473
Epoch 5/10
 - 11s - loss: 0.0494 - acc: 0.4475
Epoch 6/10
 - 11s - loss: 0.0455 - acc: 0.4477
Epoch 7/10
 - 11s - loss: 0.0414 - acc: 0.4478
Epoch 8/10
 - 11s - loss: 0.0371 - acc: 0.4478
Epoch 9/10
 - 10s - loss: 0.0339 - acc: 0.4477
Epoch 10/10
 - 11s - loss: 0.0323 - acc: 0.4482


In [78]:
data.describe

<bound method NDFrame.describe of                                                     tweet  \
0       RT @johnryan89: Oh Mané, Mané. 3 points import...   
1       Please RT!! #liverpool #liverpoolFC #LFC   "Mo...   
2       RT @JARiiseOfficial: Watching the boys @LFC he...   
3       @JackBMontgomery Scum. Here are Liverpool poli...   
4       RT @thisisanfield: "If it was me and I was in ...   
5       RT @footballpools: Players of the season so fa...   
6       Foamin I’m going to Liverpool for my 21st prac...   
7       Looking forward to getting home and watching #...   
8       RT @TheSportsman: 😏 Goals this season…\n\n- Ox...   
9       @shibani_mufc The one against Liverpool. That ...   
10      RT @GoalcomNigeria: Mohamed Salah likes new ro...   
11      @TPEassist every time I travel with TPE Liverp...   
12      RT @BIG_RIVER_CITY: .@DodgyUK\n#AcousticTour\n...   
13      RT @Jaack: Liverpool putting on Lovren after b...   
14      Betting Preview: West Ham United v Liverpoo

In [ ]:
data.describe()